In [106]:
import pandas as pd
import numpy as np

# B1

In [107]:
df=pd.read_csv(r"E:\FPT\DAT2051\Labs\Lab 7\lab7.csv")
time_serie=pd.date_range(start='01-01-2024',end='12-31-2025')
df['Date']=np.random.choice(time_serie,size=len(df))
df['Year']=df['Date'].dt.year
df['Month']=df['Date'].dt.month
df['Store']=np.random.choice(['Store A','Store B','Store C','Store D'],size=len(df))
df['Year/Month']=df['Year'].astype(str)+'-'+df['Month'].astype(str)
df.to_csv('Final_data.csv',index=False)
file_lab7='Final_data.csv'

In [ ]:
df=pd.read_csv(file_lab7)
df.head(5)

In [72]:
df.to_excel('Order.xlsx',index=False)

In [ ]:
df=pd.read_excel('Order.xlsx')
df.columns

In [ ]:
df2=df[['order_id','customer_id','total_price']]
df2.to_csv('selected_order.csv',index=False)
pd.read_csv('selected_order.csv')

# B2

In [ ]:
df=pd.read_csv(file_lab7)
df.to_hdf('hd5f_1.h5',key='df')
pd.read_hdf('hd5f_1.h5')

In [ ]:
for store in df['Store'].unique():
    store_data=df[df['Store']==store]
    for y in df['Year'].unique():
        for m in df[df['Year']==y]['Month'].unique():
            group_key=f'{store}/{y}/{m}'
            store_data_month=store_data[(store_data['Year']==y)&(store_data['Month']==m)]
            store_data_month.to_hdf('orders.h5',key=group_key,mode='a',format='table')


In [ ]:
with pd.HDFStore('orders.h5', mode='r') as store:
    print("Các nhóm dữ liệu có trong tệp HDF5:")
    print(store.keys())
    a=[ i.lstrip('/') for i in store.keys()]
pd.read_hdf('orders.h5',key=a[1])

Các nhóm dữ liệu có trong tệp HDF5:
['/Store D/2025/1', '/Store D/2025/10', '/Store D/2025/11', '/Store D/2025/12', '/Store D/2025/2', '/Store D/2025/3', '/Store D/2025/4', '/Store D/2025/5', '/Store D/2025/6', '/Store D/2025/7', '/Store D/2025/8', '/Store D/2025/9', '/Store D/2024/1', '/Store D/2024/10', '/Store D/2024/11', '/Store D/2024/12', '/Store D/2024/2', '/Store D/2024/3', '/Store D/2024/4', '/Store D/2024/5', '/Store D/2024/6', '/Store D/2024/7', '/Store D/2024/8', '/Store D/2024/9', '/Store C/2025/1', '/Store C/2025/10', '/Store C/2025/11', '/Store C/2025/12', '/Store C/2025/2', '/Store C/2025/3', '/Store C/2025/4', '/Store C/2025/5', '/Store C/2025/6', '/Store C/2025/7', '/Store C/2025/8', '/Store C/2025/9', '/Store C/2024/1', '/Store C/2024/10', '/Store C/2024/11', '/Store C/2024/12', '/Store C/2024/2', '/Store C/2024/3', '/Store C/2024/4', '/Store C/2024/5', '/Store C/2024/6', '/Store C/2024/7', '/Store C/2024/8', '/Store C/2024/9', '/Store B/2025/1', '/Store B/2025/10', 

# B3

In [131]:
chunk_size=500
a=pd.read_csv('Final_data.csv',chunksize=chunk_size)
store_total=[]
for chunk in a:
    x=chunk.groupby('Store')[['order_id','total_price']].agg({'order_id':'count','total_price':'sum'})
    x.reset_index(inplace=True)
    store_total.append(x)
f=pd.concat(store_total).groupby('Store').agg(Total_order=('order_id','sum'),Revenue=('total_price','sum')).reset_index()
f

,Store,Total_order,Revenue
0,Store A,257,127612.04
1,Store B,250,129392.31
2,Store C,251,126575.56
3,Store D,242,115749.37


In [130]:
import pandas as pd
chunk_size = 500
df=pd.read_csv(file_lab7,chunksize=chunk_size)
def process_chunk(chunk):
    summary = chunk.groupby('Store').agg(total_quantity=('quantity','sum'),revenue=('total_price','sum'))
    return summary
total_orders = 0
total_revenue = 0
for chunk in pd.read_csv('Final_data.csv',chunksize=chunk_size):
    chunk_summary = process_chunk(chunk)
    print(chunk_summary)
    total_orders += chunk_summary['total_quantity'].sum()
    total_revenue += chunk_summary['revenue'].sum()
print(total_orders)
print(total_revenue)

         total_quantity   revenue
Store                            
Store A             659  60466.82
Store B             699  66027.27
Store C             652  66370.78
Store D             682  59934.14
         total_quantity   revenue
Store                            
Store A             778  67145.22
Store B             645  63365.04
Store C             760  60204.78
Store D             638  55815.23
5513
499329.28


# B 4

In [138]:
import dask.dataframe as dd
ddf = dd.read_csv(file_lab7)
ddf['quantity'].sum().compute()

5513